In [ ]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Define game constants
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
FPS = 60

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Initialize the game window
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Memory Game")

clock = pygame.time.Clock()

# Load gesture images or emojis (replace with your own images)
gesture_images = {
    "A": pygame.image.load("gesture_a.jpg"),
    "B": pygame.image.load("gesture_b.jpg"),
    "C": pygame.image.load("gesture_c.jpg"),
    # Add more gestures as needed
}

# Resize gesture images to the same size
GESTURE_SIZE = (200, 200)
for gesture in gesture_images:
    gesture_images[gesture] = pygame.transform.scale(gesture_images[gesture], GESTURE_SIZE)

# Game settings
TIMEOUT = 1000  # Time delay between gestures in milliseconds

# Game variables
round_number = 1
score = 0
sequence = []
player_sequence = []

# Fonts
font = pygame.font.SysFont(None, 48)

# Slot variables
slot_x = (WINDOW_WIDTH - GESTURE_SIZE[0]) // 2
slot_y = (WINDOW_HEIGHT - GESTURE_SIZE[1]) // 2

# Instruction texts
watch_text = font.render("Watch and remember the sequence", True, BLACK)
repeat_text = font.render("Repeat the sequence", True, BLACK)

# Game state variables
displaying_sequence = True
waiting_for_input = False
current_index = 0

# Placeholder variables
num_underlines = {2: 2, 4: 4}
current_underlines = 0
correct_guesses = 0
placeholder_color = WHITE
placeholders = ""

# Additional key press requirement variables
additional_key_required = False

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                running = False
            if event.key == pygame.K_a:
                if waiting_for_input and not additional_key_required:  # Check if additional key is required
                    player_sequence.append("A")
                    current_underlines += 1
                    placeholders += "A "
                    if len(player_sequence) == round_number and len(player_sequence) == len(sequence):
                        additional_key_required = True
            if event.key == pygame.K_b:
                if waiting_for_input and not additional_key_required:  # Check if additional key is required
                    player_sequence.append("B")
                    current_underlines += 1
                    placeholders += "B "
                    if len(player_sequence) == round_number and len(player_sequence) == len(sequence):
                        additional_key_required = True
            if event.key == pygame.K_c:
                if waiting_for_input and not additional_key_required:  # Check if additional key is required
                    player_sequence.append("C")
                    current_underlines += 1
                    placeholders += "C "
                    if len(player_sequence) == round_number and len(player_sequence) == len(sequence):
                        additional_key_required = True
            if additional_key_required and event.key == pygame.K_RETURN:
                additional_key_required = False
                correct_guesses = sum(1 for i in range(len(player_sequence)) if player_sequence[i] == sequence[i])
                score += correct_guesses

                if len(player_sequence) == round_number:
                    gesture = random.choice(list(gesture_images.keys()))
                    sequence.append(gesture)
                    round_number += 1
                    player_sequence = []
                    displaying_sequence = True
                    waiting_for_input = False
                    current_underlines = 0
                    placeholders = ""

    window.fill(WHITE)

    # Display round number and score
    round_text = font.render(f"Round: {round_number}", True, BLACK)
    score_text = font.render(f"Score: {score}", True, BLACK)
    window.blit(round_text, (20, 20))
    window.blit(score_text, (WINDOW_WIDTH - score_text.get_width() - 20, 20))

    if displaying_sequence:
        # Display "Watch and remember the sequence" text
        window.blit(watch_text, (WINDOW_WIDTH // 2 - watch_text.get_width() // 2, 100))

        if len(sequence) == 0:
            # Generate new gesture for the sequence if the sequence is empty
            for _ in range(round_number):
                gesture = random.choice(list(gesture_images.keys()))
                sequence.append(gesture)

        if current_index < len(sequence):
            gesture = sequence[current_index]
            gesture_image = gesture_images[gesture]
            x = slot_x
            y = slot_y
            window.blit(gesture_image, (x, y))
            index_text = font.render(str(current_index + 1), True, BLACK)
            window.blit(index_text, (x + GESTURE_SIZE[0] // 2 - index_text.get_width() // 2, y - 30))

            pygame.display.update()
            pygame.time.delay(TIMEOUT)
            window.fill(WHITE)
            current_index += 1

        else:
            displaying_sequence = False
            waiting_for_input = True
            current_index = 0

    elif waiting_for_input:
        # Display "Repeat the sequence" text
        window.blit(repeat_text, (WINDOW_WIDTH // 2 - repeat_text.get_width() // 2, 100))

        if len(player_sequence) == len(sequence):
            if additional_key_required:
                additional_text = font.render("Press ENTER to continue", True, BLACK)
                window.blit(additional_text, (WINDOW_WIDTH // 2 - additional_text.get_width() // 2, 200))
            else:
                correct_guesses = sum(1 for i in range(len(player_sequence)) if player_sequence[i] == sequence[i])
                score += correct_guesses

                if len(player_sequence) == round_number and len(player_sequence) == len(sequence):
                    gesture = random.choice(list(gesture_images.keys()))
                    sequence.append(gesture)
                    round_number += 1
                    player_sequence = []
                    displaying_sequence = True
                    waiting_for_input = False
                    current_underlines = 0
                    placeholders = ""


    if not displaying_sequence and waiting_for_input:
        num_placeholder = num_underlines.get(round_number, 0)
        letter_width = font.size(" ")[0]
        total_width = len(player_sequence) * (letter_width + 25) - 25  # Adjusted width

        placeholders_width = num_placeholder * (letter_width + 25) - 25  # Adjusted width

        placeholder_x = (slot_x + (WINDOW_WIDTH - placeholders_width) // 2)/2 + 60
        start_x = placeholder_x + (placeholders_width - total_width) // 2
        placeholder_y = slot_y + GESTURE_SIZE[1] + 30

        for i in range(len(player_sequence)):
            if player_sequence[i] == sequence[i]:
                color = GREEN
            else:
                color = RED

            letter_surface = font.render(player_sequence[i], True, color)
            letter_x = start_x + i * (letter_width + 25)  # Adjusted position
            letter_y = placeholder_y
            window.blit(letter_surface, (letter_x, letter_y))

    pygame.display.update()
    clock.tick(FPS)

# Quit the game
pygame.quit()


pygame 2.5.0 (SDL 2.28.0, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
